In [1]:
import cv2
import numpy as np

def roi(img, coordinates):
    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, coordinates, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def lines(img, hough_transform):
    line_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if hough_transform is not None:
        for line in hough_transform:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), thickness=20)
    else:
        print("No lines detected!")
    img = cv2.addWeighted(img, 0.8, line_image, 1, 0.0)
    return img

def process(img):
    height, width = img.shape[:2]


    region_of_interest_coordinates = [
    (width // 20, height // 2),
    (19 * width // 20, height // 2),
    (width, height),
    (0, height)
]
 



    gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray_scale, 100, 200)

    # Mask the entire frame
    masked_image = roi(canny, np.array([region_of_interest_coordinates], np.int32))

    hough_transform = cv2.HoughLinesP(
        masked_image,
        rho=6,
        theta=np.pi / 60,
        threshold=160,
        lines=np.array([]),
        minLineLength=40,
        maxLineGap=25
    )

    return lines(img, hough_transform)

cap = cv2.VideoCapture('testing.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video ended or cannot read frame.")
        break

    processed_frame = process(frame)
    cv2.imshow("Lane Detection (Full ROI)", processed_frame)

    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
